# Project

## Import libraries

In [1]:
#!pip install numpy==1.21.4
#!pip install pandas==1.3.4
#!pip install keras==2.1.6
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


## Download and preprocess data

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


The first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 


In [3]:
print('The data has {} datapoints'.format(concrete_data.shape))

The data has (1030, 9) datapoints


In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


#### Check if there is missing data

In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

### Split data into predictors and target

In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except strength
target = concrete_data['Strength']

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

### Normalize the data - subtract mean and divide by standard deviation

In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [10]:
#Save number of columns
n_cols = predictors_norm.shape[1]
n_cols

8

In [11]:
import keras
from keras.models import Sequential 
from keras.layers import Dense

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

## Build neural network

Define regression model

In [12]:
def regression_model():
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    model.compile(optimizer='adam', loss= 'mean_squared_error')
    return model

## Train the Network

In [34]:
#build the model
model = regression_model()

In [35]:
mse_list = []

for _ in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=40)
    print('Training data.shape=', X_train.shape, 'Training Labels shape=', y_train.shape)
    print('Testing data shape=', X_test.shape, ' Testing Labels shape=', y_test.shape)

    model.fit(X_train, y_train, epochs=100, verbose=0)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_list.append(mse)

mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)

print("Mean of Mean Squared Errors:", mean_mse)
print("Standard Deviation of Mean Squared Errors:", std_mse)


Training data.shape= (721, 8) Training Labels shape= (721,)
Testing data shape= (309, 8)  Testing Labels shape= (309,)
Training data.shape= (721, 8) Training Labels shape= (721,)
Testing data shape= (309, 8)  Testing Labels shape= (309,)
Training data.shape= (721, 8) Training Labels shape= (721,)
Testing data shape= (309, 8)  Testing Labels shape= (309,)
Training data.shape= (721, 8) Training Labels shape= (721,)
Testing data shape= (309, 8)  Testing Labels shape= (309,)
Training data.shape= (721, 8) Training Labels shape= (721,)
Testing data shape= (309, 8)  Testing Labels shape= (309,)
Training data.shape= (721, 8) Training Labels shape= (721,)
Testing data shape= (309, 8)  Testing Labels shape= (309,)
Training data.shape= (721, 8) Training Labels shape= (721,)
Testing data shape= (309, 8)  Testing Labels shape= (309,)
Training data.shape= (721, 8) Training Labels shape= (721,)
Testing data shape= (309, 8)  Testing Labels shape= (309,)
Training data.shape= (721, 8) Training Labels sh

_Observation_

When using 100 epochs and 3 layer neural network with 10 nodes each:
    
Mean of Mean Squared Errors: 43.00403287124882

Standard Deviation of Mean Squared Errors: 7.483875765364484

### Choosing the number of hidden layers
Using the rule of thumb number of hidden layers should be close to (inputs+outputs) * 2/3, I chose to use 6 hidden layers

In [38]:
def regression_model2():
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    model.compile(optimizer='adam', loss= 'mean_squared_error')
    return model

In [39]:
model2 = regression_model2()

In [42]:
mse_list = []

X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=40)
print('Training data.shape=', X_train.shape, 'Training Labels shape=', y_train.shape)
print('Testing data shape=', X_test.shape, ' Testing Labels shape=', y_test.shape)

for _ in range(50):

    model2.fit(X_train, y_train, epochs=100, verbose=0)

    y_pred = model2.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_list.append(mse)

mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)

print("Mean of Mean Squared Errors:", mean_mse)
print("Standard Deviation of Mean Squared Errors:", std_mse)


Training data.shape= (721, 8) Training Labels shape= (721,)
Testing data shape= (309, 8)  Testing Labels shape= (309,)
Mean of Mean Squared Errors: 38.073708632343894
Standard Deviation of Mean Squared Errors: 3.7104461046908717


_Observation_

Mean of Mean Squared Errors: 38.073708632343894

Standard Deviation of Mean Squared Errors: 3.7104461046908717

### Increase the number of nodes per hidden layer

In [49]:
def regression_model3():
    model = Sequential()
    model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    model.compile(optimizer='adam', loss= 'mean_squared_error')
    return model

In [50]:
model3 = regression_model3()
mse_list = []

X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=40)
print('Training data.shape=', X_train.shape, 'Training Labels shape=', y_train.shape)
print('Testing data shape=', X_test.shape, ' Testing Labels shape=', y_test.shape)

for _ in range(50):

    model3.fit(X_train, y_train, epochs=100, verbose=0)

    y_pred = model3.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_list.append(mse)

mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)

print("Mean of Mean Squared Errors:", mean_mse)
print("Standard Deviation of Mean Squared Errors:", std_mse)


Training data.shape= (721, 8) Training Labels shape= (721,)
Testing data shape= (309, 8)  Testing Labels shape= (309,)
Mean of Mean Squared Errors: 28.333912386021094
Standard Deviation of Mean Squared Errors: 1.840202966910573


_Observation_

The Mean of Mean Squared Errors has dropped by approximately 10 units to 28.333912386021094


### Futher increase the number of nodes per layer
Doubled the number of nodes per layer to 100

In [20]:
def regression_model4():
    model = Sequential()
    model.add(Dense(100, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1))
    
    model.compile(optimizer='adam', loss= 'mean_squared_error')
    return model

In [13]:
model4 = regression_model4()
mse_list = []

X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=40)
print('Training data.shape=', X_train.shape, 'Training Labels shape=', y_train.shape)
print('Testing data shape=', X_test.shape, ' Testing Labels shape=', y_test.shape)

for _ in range(50):

    model4.fit(X_train, y_train, epochs=100, verbose=0)

    y_pred = model4.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_list.append(mse)

mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)

print("Mean of Mean Squared Errors:", mean_mse)
print("Standard Deviation of Mean Squared Errors:", std_mse)






Training data.shape= (721, 8) Training Labels shape= (721,)
Testing data shape= (309, 8)  Testing Labels shape= (309,)




2023-06-13 16:17:44.464405: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2023-06-13 16:17:44.470288: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2394310000 Hz
2023-06-13 16:17:44.471146: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55e22de3a6b0 executing computations on platform Host. Devices:
2023-06-13 16:17:44.471223: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2023-06-13 16:17:44.654722: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line fl

Mean of Mean Squared Errors: 23.83200302637062
Standard Deviation of Mean Squared Errors: 1.273735259233312


_Observation_

Doubling the number of nodes per layer to 100 improved the model's accuracy from 28.333912386021094 to 23.83200302637062

Accuracy improved but rate of improvement in the model's accuracy is starting to lower with the increase in the number if nodes. 

## Increse the number of layers as well
Increased the number of layers to 7

In [14]:
def regression_model5():
    model = Sequential()
    model.add(Dense(100, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1))
    
    model.compile(optimizer='adam', loss= 'mean_squared_error')
    return model

In [15]:
model5 = regression_model5()
mse_list = []
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=40)
print('Training data.shape=', X_train.shape, 'Training Labels shape=', y_train.shape)
print('Testing data shape=', X_test.shape, ' Testing Labels shape=', y_test.shape)

for _ in range(50):

    model5.fit(X_train, y_train, epochs=100, verbose=0)

    y_pred = model5.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_list.append(mse)

mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)

print("Mean of Mean Squared Errors:", mean_mse)
print("Standard Deviation of Mean Squared Errors:", std_mse)


Training data.shape= (721, 8) Training Labels shape= (721,)
Testing data shape= (309, 8)  Testing Labels shape= (309,)
Mean of Mean Squared Errors: 26.591513116933896
Standard Deviation of Mean Squared Errors: 1.3615064304756817


_Observation_

Mean of Mean Squared Errors: 26.591513116933896

Standard Deviation of Mean Squared Errors: 1.3615064304756817

Increasing the number of layers beyond 6 does not improve the accuracy

### Decrease the hidden layers

In [16]:
def regression_model6():
    model = Sequential()
    model.add(Dense(100, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1))
    
    model.compile(optimizer='adam', loss= 'mean_squared_error')
    return model

In [17]:
model4 = regression_model6()
mse_list = []
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=40)
print('Training data.shape=', X_train.shape, 'Training Labels shape=', y_train.shape)
print('Testing data shape=', X_test.shape, ' Testing Labels shape=', y_test.shape)

for _ in range(50):

    model4.fit(X_train, y_train, epochs=100, verbose=0)

    y_pred = model4.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_list.append(mse)

mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)

print("Mean of Mean Squared Errors:", mean_mse)
print("Standard Deviation of Mean Squared Errors:", std_mse)


Training data.shape= (721, 8) Training Labels shape= (721,)
Testing data shape= (309, 8)  Testing Labels shape= (309,)
Mean of Mean Squared Errors: 26.373374888555258
Standard Deviation of Mean Squared Errors: 1.1181505655426052


_Observation_

Mean of Mean Squared Errors: 

Standard Deviation of Mean Squared Errors:

Decreasing the number of layers to 5 does not improve the accuracy

## Further increse number of nodes
Increase the number of nodes to by 50 to 150

In [18]:
def regression_model4():
    model = Sequential()
    model.add(Dense(150, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(150, activation='relu'))
    model.add(Dense(150, activation='relu'))
    model.add(Dense(150, activation='relu'))
    model.add(Dense(150, activation='relu'))
    model.add(Dense(150, activation='relu'))
    model.add(Dense(1))
    
    model.compile(optimizer='adam', loss= 'mean_squared_error')
    return model

In [19]:
model4 = regression_model4()
mse_list = []
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=40)
print('Training data.shape=', X_train.shape, 'Training Labels shape=', y_train.shape)
print('Testing data shape=', X_test.shape, ' Testing Labels shape=', y_test.shape)

for _ in range(50):

    model4.fit(X_train, y_train, epochs=100, verbose=0)

    y_pred = model4.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_list.append(mse)

mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)

print("Mean of Mean Squared Errors:", mean_mse)
print("Standard Deviation of Mean Squared Errors:", std_mse)


Training data.shape= (721, 8) Training Labels shape= (721,)
Testing data shape= (309, 8)  Testing Labels shape= (309,)
Mean of Mean Squared Errors: 24.269676917098867
Standard Deviation of Mean Squared Errors: 1.8779429784943973


_Observation_

Mean of Mean Squared Errors: 24.269676917098867

Standard Deviation of Mean Squared Errors: 1.8779429784943973
Increasing the number of nodes per layer to 150 from 100 does not improve the accuracy

## Conclusion
* The optimal number of hidden layers is (inputs+outputs) * 2/3 = 6
* The optimal number of nodes per layer is 100

Model 4 is the most accurate and sparse model with:

Mean of Mean Squared Errors: 23.83200302637062  

Standard Deviation of Mean Squared Errors: 1.273735259233312